# ETL with DataFrames

#### Replace all \*\*\*TODO\*\*\* placeholders with proper Spark API function calls. 
#### You can use code completion with "TAB" to display the list of available functions
#### Use Escape + H to see list of all the Jupyter commands and shortcuts

### Import necessary functions and types

In [ ]:
from pyspark.sql import HiveContext
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType, StringType, IntegerType
import sys

### Define UDF needed to clean the data

In [ ]:
def isDurationCorrect(duration):
    return duration is not None and duration != 'INF' and 30 <= int(duration) and int(duration) <= 1200

def removeBrackets(text):
    return text[1:-1]

### Register UDFs so they can be used in SQL statements

In [ ]:
udfIsDurationCorrect=udf(isDurationCorrect, BooleanType())
udfRemoveBrackets=udf(removeBrackets, StringType())

### Define variables

In [ ]:
database = "<your database here>"

### Load raw dataset from Hive

In [ ]:
logs = sqlContext.***TODO***("SELECT * FROM %s.logs" % database)
logs.show(5)
logs.count()

### Leave only SongPlayed events

In [ ]:
streams_raw = logs.***TODO***(logs['eventType'] == 'SongPlayed')
streams_raw.show(5)
streams_raw.count()

### Remove unnecessary 'eventType' column from dataset

In [ ]:
streams_projected = streams_raw.***TODO***('eventType')
streams_projected.show(5)

### Rename 'itemId' column into 'trackId'

In [ ]:
streams = streams_projected.withColumnRenamed('itemId', 'trackId')
streams.show(5)

### Exclude records with incorrect value in 'duration' field 

In [ ]:
streams_correct = streams.***TODO***(udfIsDurationCorrect('duration'))
streams_correct.show(5)
streams_correct.count()

### Remove brackets around 'ts' field and cast 'duration' field to integer type

In [ ]:
streams_cleaned = streams_correct.***TODO***(udfRemoveBrackets('ts').alias('ts'),
                                             'host', 'userId', 'trackId', 
                                             streams_correct['duration'].cast('int'))
streams_cleaned.show(5)

### Remove duplicates from dataset

In [ ]:
streams_unique = streams_cleaned.***TODO***()
streams_unique.count()

### Store cleaned dataset in Hive table

In [ ]:
streams_unique.write.***TODO***("%s.stream" % database, mode="overwrite")